In [10]:
import pandas as pd
import os
import plotly.express as px

In [11]:
in_relative_path = "../../../data/gold/"
in_absolute_path = os.path.abspath(os.path.join(os.getcwd(), in_relative_path))

data = pd.read_csv(f"{in_absolute_path}/rte_daily_2014_2024.csv")
data["Date"] = pd.to_datetime(data["Date"])


In [12]:
data.dtypes

Date             datetime64[ns]
Nature                   object
type_tempo               object
Consommation            float64
Prévision_J-1           float64
Prévision_J             float64
Fioul                   float64
Charbon                 float64
Gaz                     float64
Nucléaire               float64
Eolien                  float64
Solaire                 float64
Hydraulique             float64
Pompage                 float64
Bioénergies             float64
Ech_physiques           float64
Taux_de_Co2             float64
Ech_comm                float64
sun                     float64
wind                    float64
dtype: object

In [13]:
fig = px.line(data, x="Date", y="Consommation")
fig.show()

In [14]:
fig = px.line(data, x="Date", y="sun")
fig.show()